In [ ]:
# hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp right_menu_widget

In [ ]:
# hide
from nbdev import *

In [ ]:
#exporti
from ipywidgets import HBox, Dropdown, Layout, VBox, Checkbox
from functools import partial
from typing import List
from ipyannotator.custom_input.coordinates import CoordinateInput
from ipyannotator.mltypes import BboxCoordinate, BboxVideoCoordinate
from ipyannotator.custom_input.buttons import ActionButton

In [ ]:
#exporti

class BBoxItem(VBox):
    """BBox row with select button, coordinates inputs and delete button"""

    def __init__(
        self,
        bbox_coord: BboxCoordinate,
        max_coord_input_values: BboxCoordinate,
        index: int,
        options: List[str] = None
    ):
        super().__init__()

        self.bbox_coord = bbox_coord
        self.index = index
        self._max_coord_input_values = max_coord_input_values
        self.layout = Layout(display='flex', overflow='hidden')
        self.btn_delete = self._btn_delete(index)
        self.dropdown_classes = self._dropdown_classes(options)
        self.btn_select = self._btn_select(index)
        self.input_coordinates = self._coordinate_inputs(bbox_coord)

        self.children = [
            HBox([
                self.btn_select,
                self.dropdown_classes,
                self.input_coordinates,
                self.btn_delete
            ])
        ]

    def _btn_delete(self, index: int) -> ActionButton:
        return ActionButton(
            layout=Layout(width='auto'),
            icon="trash",
            button_style="danger",
            value=index
        )

    def _dropdown_classes(self, options: list, value: str = None) -> Dropdown:
        return Dropdown(
            layout=Layout(width='auto'),
            options=options,
            value=value
        )

    def _btn_select(self, index: int) -> ActionButton:
        return ActionButton(
            icon="mouse-pointer",
            layout=Layout(width='auto'),
            value=index
        )

    def _coordinate_inputs(self, bbox_coord: BboxCoordinate):
        return CoordinateInput(
            bbox_coord=bbox_coord,
            input_max=self._max_coord_input_values
        )

In [ ]:
#hide

bbox_item = BBoxItem(
    bbox_coord=BboxCoordinate(*[10, 10, 10, 10]),
    index=0,
    options=['Item 01', 'Item 02'],
    max_coord_input_values=BboxCoordinate(*[10, 10, 10, 10])
)
bbox_item

In [ ]:
#exporti

class BBoxVideoItem(BBoxItem):
    def __init__(
        self,
        bbox_coord: BboxVideoCoordinate,
        index: int,
        label: List[str],
        options: List[str],
        selected: bool = False,
        btn_delete_enabled: bool = True
    ):
        super(VBox, self).__init__()

        self.selected = selected
        self.bbox_coord = bbox_coord
        self.object_checkbox = self._object_checkbox()
        self.btn_select = self._btn_select(index)
        self.btn_delete = self._btn_delete(index)
        self.dropdown_classes = self._dropdown_classes(
            options=options,
            value=label[0] if label else None
        )

        self._options = [
            self.object_checkbox,
            self.btn_select,
            self.dropdown_classes,
        ]

        if btn_delete_enabled:
            self._options.append(self.btn_delete)

        self.children = [
            HBox(self._options)
        ]

    def _object_checkbox(self) -> Checkbox:
        return Checkbox(
            value=self.selected,
            indent=False,
            description=str(self.bbox_coord.id),
            layout=Layout(width='auto')
        )

In [ ]:
#hide

bbox_item = BBoxVideoItem(
    bbox_coord=BboxVideoCoordinate(*[10, 10, 10, 10, 'Object 01']),
    index=0,
    label=['Item 01'],
    options=['Item 01', 'Item 02']
)
bbox_item

In [ ]:
#exporti

class BBoxList(VBox):
    """Render the list of bbox items and set the interactions"""

    def __init__(
        self,
        classes: list,
        max_coord_input_values: BboxCoordinate,
        on_coords_changed: callable,
        on_label_changed: callable,
        on_btn_delete_clicked: callable,
        on_btn_select_clicked: callable
    ):
        super().__init__()
        self._classes = classes
        self._max_coord_input_values = max_coord_input_values
        self._on_coords_changed = on_coords_changed
        self._on_btn_delete_clicked = on_btn_delete_clicked
        self._on_label_changed = on_label_changed
        self._on_btn_select_clicked = on_btn_select_clicked

    def render_btn_list(self, bbox_coords: List[BboxCoordinate], classes: List[List[str]]):
        elements = []
        num_children = len(self.children)

        for index, coord in enumerate(bbox_coords[num_children:], num_children):
            bbox_item = BBoxItem(
                index=index,
                options=self._classes,
                bbox_coord=coord,
                max_coord_input_values=self._max_coord_input_values,
            )

            bbox_item.btn_delete.on_click(self.del_element)
            bbox_item.input_coordinates.uuid = index
            bbox_item.input_coordinates.coord_changed = self._on_coords_changed
            bbox_item.btn_select.on_click(self._on_btn_select_clicked)
            if classes and classes[index]:
                bbox_item.dropdown_classes.value = classes[index][0] or None
            bbox_item.dropdown_classes.observe(
                partial(self._on_label_changed, index=index),
                names="value",
            )

            elements.append(bbox_item)

        self.children = [*list(self.children), *elements]

    def clear(self):
        self.children = []

    def _update_bbox_list_index(self, elements: list, index: int):
        for index, element in enumerate(elements[index:], index):
            # updates select btn
            element.children[0].value = index
            # update label dropdown
            dropdown = element.children[0].children[1]
            dropdown.unobserve_all()
            dropdown.observe(
                partial(self._on_label_changed, index=index),
                names="value"
            )
            # update inputs
            element.children[0].children[2].uuid = index
            # updates delete btn
            element.children[0].children[-1].value = index

    def del_element(self, btn: ActionButton):
        index = btn.value
        elements = list(self.children)
        del elements[index]
        self._update_bbox_list_index(elements, index)
        self.children = elements
        self._on_btn_delete_clicked(index)

In [ ]:
#hide

# it can render bbox list

# just for test purposes
def f(x):
    return x


bbox_list = BBoxList(['A', 'B'], BboxCoordinate(*[20, 30, 40, 50]), f, f, f, f)

classes = [[], [], []]
bbox = [
    {'x': 10, 'y': 10, 'width': 20, 'height': 30},
    {'x': 20, 'y': 30, 'width': 10, 'height': 10},
    {'x': 30, 'y': 30, 'width': 10, 'height': 10}
]

bbox = [BboxCoordinate(**b) for b in bbox]

bbox_list.render_btn_list(bbox, classes)

bbox_list

In [ ]:
#exporti

class BBoxVideoList(BBoxList):
    def __init__(
        self,
        classes: list,
        on_label_changed: callable,
        on_btn_delete_clicked: callable,
        on_btn_select_clicked: callable,
        on_checkbox_object_clicked: callable,
        btn_delete_enabled: bool = True
    ):
        super().__init__(
            classes=classes,
            max_coord_input_values=None,
            on_coords_changed=None,
            on_label_changed=on_label_changed,
            on_btn_delete_clicked=on_btn_delete_clicked,
            on_btn_select_clicked=on_btn_select_clicked
        )
        self.bbox_coords_selected = []
        self._btn_delete_enabled = btn_delete_enabled
        self._on_checkbox_object_clicked = on_checkbox_object_clicked

    def __getitem__(self, index: int):
        return self.children[index]

    def render_btn_list(
        self,
        bbox_coords: List[BboxVideoCoordinate],
        classes: list,
        labels: List[List[str]],
        selected: List[int] = []
    ):
        elements = []
        num_children = len(self.children)

        for index, bbox_coord in enumerate(bbox_coords[num_children:], num_children):
            bbox_item = BBoxVideoItem(
                index=index,
                options=self._classes,
                bbox_coord=bbox_coord,
                label=labels[index],
                selected=index in selected,
                btn_delete_enabled=self._btn_delete_enabled
            )

            bbox_item.btn_delete.on_click(self.del_element)
            bbox_item.btn_select.on_click(self._on_btn_select_clicked)

            if classes and classes[index]:
                bbox_item.dropdown_classes.value = classes[index][0] or None

            bbox_item.dropdown_classes.observe(
                partial(self._on_label_changed, index=index),
                names="value",
            )

            bbox_item.object_checkbox.observe(
                partial(self._on_checkbox_object_clicked, index=index, bbox_coord=bbox_coord),
                names="value",
            )

            elements.append(bbox_item)

        self.children = [*list(self.children), *elements]

    def clear(self):
        self.children = []

    def _update_bbox_list_index(self, elements: list, index: int):
        for index, element in enumerate(elements[index:], index):
            #updates checkbox
            checkbox = element.children[0].children[0]
            checkbox.unobserve_all()
            checkbox.observe(
                partial(self._on_checkbox_object_clicked, index=index),
                names="value",
            )
            # updates select btn
            element.children[1].children[0].value = index
            # update label dropdown
            dropdown = element.children[1].children[1]
            dropdown.unobserve_all()
            dropdown.observe(
                partial(self._on_label_changed, index=index),
                names="value"
            )
            # updates delete btn
            element.children[1].children[-1].value = index

    def del_element(self, btn: ActionButton):
        index = btn.value
        elements = list(self.children)
        del elements[index]
        self._update_bbox_list_index(elements, index)
        self.children = elements
        self._on_btn_delete_clicked(index)

In [ ]:
#hide

# it can render bbox list

# just for test purposes
def f(x):
    return x


bbox_list = BBoxVideoList(['A', 'B'], f, f, f, f)

classes = [[], [], []]
bbox = [
    BboxVideoCoordinate(**{'x': 10, 'y': 10, 'width': 20, 'height': 30, 'id': 'Object1'}),
    BboxVideoCoordinate(**{'x': 20, 'y': 30, 'width': 10, 'height': 10, 'id': 'Object2'}),
    BboxVideoCoordinate(**{'x': 30, 'y': 30, 'width': 10, 'height': 10, 'id': 'Object3'})
]
labels = [['A'], ['B'], ['A']]

bbox_list.render_btn_list(bbox_coords=bbox, classes=classes, labels=labels)

bbox_list

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()